In [ ]:
import os
import cv2
import glob
import ntpath
import random
import numpy as np
import pandas as pd

In [ ]:
for i in range(10):
    
    predict_nucleus_grade(i+1)

In [ ]:
def predict_nucleus_grade(split_rule = 1):

    csv_target_path = "G:/LiverCancer/predict_result/" + str(split_rule) + "_result.csv"

    model_path = f"G:/LiverCancer/model/model_liver_CNN_{split_rule}__best.hd5"
    
    model = get_net(load_weight_path=model_path)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    src_path = "G:/LiverCancer/resources/split_rule/" + str(split_rule) +"/"
    test_biopsy_names = pd.read_csv(src_path + "holdout.csv")["file_path"].apply(ntpath.basename).tolist()

    src_path = "G:/LiverCancer/nucleus/image/"
    nucleus_samples = [file for fold_1 in os.listdir(src_path)
                            for fold_2 in os.listdir(os.path.join(src_path, fold_1))
                            for file in glob.glob(os.path.join(src_path, fold_1, fold_2, "*.jpg"))]

    test_nucleus_samples = [s for s in nucleus_samples if (s.split(".jpg")[0][-9:]+".jpg") in test_biopsy_names]

    src_path = "G:/LiverCancer/masked_biopsy_resized/"
    biopsy_masks = [file_path for fold in os.listdir(src_path) for file_path in glob.glob(os.path.join(src_path, fold, "*.jpg"))]

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    batch_counter, batch_size, info_list, image_list, mask_list, labels = 0, 32, [], [], [], []

    for index, sample_path in enumerate(test_nucleus_samples):

        print(index)
    
        temp = ntpath.basename(sample_path).split("_")
        if temp[0]+temp[1] == "grade0": biopsy_mask_from = "ImageN."+temp[2]+".jpg"
        if temp[0]+temp[1] == "grade1": biopsy_mask_from = "Image1-"+temp[2]+".jpg"
        if temp[0]+temp[1] == "grade2": biopsy_mask_from = "Image2-"+temp[2]+".jpg"
        if temp[0]+temp[1] == "grade3": biopsy_mask_from = "Image3-"+temp[2]+".jpg"
        if temp[0]+temp[1] == "grade4": biopsy_mask_from = "Image4-"+temp[2]+".jpg"
            
        for biopsy_mask in biopsy_masks:
            
            if biopsy_mask_from in biopsy_mask: biopsy_mask_from = biopsy_mask
        
        if "grade0" in sample_path: sample_label = 0
        if "grade1" in sample_path: sample_label = 1
        if "grade2" in sample_path: sample_label = 2
        if "grade3" in sample_path: sample_label = 3
        if "grade4" in sample_path: sample_label = 4

        info_list.append((sample_path, biopsy_mask_from, sample_label))
        
        sample_image = cv2.imread(sample_path)
        sample_image = (sample_image - np.average(sample_image)) / np.std(sample_image)
        sample_image = sample_image.reshape(1, sample_image.shape[0], sample_image.shape[1], 3)
        image_list.append(sample_image)
        
        mask_image = cv2.imread(biopsy_mask_from)
        indent_x = random.randint(0, 128)
        indent_y = random.randint(0, 128)       
        mask_image = mask_image[indent_x:indent_x + 128, indent_y:indent_y + 128]
        mask_image = (mask_image - np.average(mask_image)) / np.std(mask_image)
        mask_image = mask_image.reshape(1, mask_image.shape[0], mask_image.shape[1], 3)
        mask_list.append(mask_image)
   
        batch_counter += 1

        if len(image_list)% batch_size==0 or batch_counter==len(test_nucleus_samples):

            x_sample = np.vstack(image_list); x_mask = np.vstack(mask_list)
            
            p = model.predict({"sample_image":x_sample, "mask_image":x_mask}, batch_size=batch_size)

            for i in range(len(p)):

                sample_path = info_list[i][0]
                sample_label = info_list[i][2]

                labels.append([sample_path,str(p[i][0]),str(p[i][1]),str(p[i][2]),str(p[i][3]),str(p[i][4]), sample_label])

            info_list, image_list, mask_list = [], [], []    

    df_labels = pd.DataFrame(labels, columns=["sample_path", "grade_p_0", "grade_p_1", "grade_p_2", "grade_p_3", "grade_p_4","sample_label"])
    df_labels.to_csv(csv_target_path, index=False)

In [ ]:
from keras.models import Model
from keras.optimizers import SGD
from keras.layers.advanced_activations import ReLU
from keras.metrics import categorical_accuracy, categorical_crossentropy
from keras.layers import Input,AveragePooling2D,Convolution2D,BatchNormalization,MaxPooling2D,Concatenate,GlobalMaxPooling2D,Dense,concatenate

def get_net(input_shape=(128, 128, 3), load_weight_path=None) -> Model:  
    
    inputs = Input(shape=input_shape, name="sample_image")
    inputs_mask = Input(shape=input_shape, name="mask_image")
    x = inputs
    x_mask = inputs_mask
    
    ##################################################################################################################
    x_ident_1 = x
    x_ident_1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_ident_1)
    # 1st layer group
    x = Convolution2D(16, 3, 3, activation=None, border_mode='same', name='conv1a', subsample=(1, 1))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Convolution2D(16, 3, 3, activation=None, border_mode='same', name='conv1b', subsample=(1, 1))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool1')(x)
    x = Concatenate(axis=3)([x,x_ident_1])
    
    x_ident_1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_ident_1)
    x_ident_2 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x)
    # 2nd layer group
    x = Convolution2D(32, 3, 3, activation=None, border_mode='same', name='conv2a', subsample=(1, 1))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Convolution2D(32, 3, 3, activation=None, border_mode='same', name='conv2b', subsample=(1, 1))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool2')(x)
    x = Concatenate(axis=3)([x,x_ident_1,x_ident_2])

    x_ident_1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_ident_1)
    x_ident_2 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_ident_2)
    x_ident_3 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x)
    # 3rd layer group
    x = Convolution2D(64, 3, 3, activation=None, border_mode='same', name='conv3a', subsample=(1, 1))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Convolution2D(64, 3, 3, activation=None, border_mode='same', name='conv3b', subsample=(1, 1))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool3')(x)
    x = Concatenate(axis=3)([x,x_ident_1,x_ident_2,x_ident_3])
     
    x_ident_1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_ident_1)
    x_ident_2 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_ident_2)
    x_ident_3 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_ident_3)
    x_ident_4 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x)
    # 4th layer group
    x = Convolution2D(128, 3, 3, activation=None, border_mode='same', name='conv4a', subsample=(1, 1),)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Convolution2D(128, 3, 3, activation=None, border_mode='same', name='conv4b', subsample=(1, 1),)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool4')(x)
    x = Concatenate(axis=3)([x,x_ident_1,x_ident_2,x_ident_3,x_ident_4])
    
    x = GlobalMaxPooling2D()(x)
    x = BatchNormalization(name="final_features_344")(x)
    
    ##################################################################################################################
    x_mask_ident_1 = x_mask
    x_mask_ident_1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_mask_ident_1)
    # 1st layer group
    x_mask = Convolution2D(8, 3, 3, activation=None, border_mode='same', name='conv1_mask', subsample=(1, 1))(x_mask)
    x_mask = BatchNormalization()(x_mask)
    x_mask = ReLU()(x_mask)
    x_mask = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool1_mask')(x_mask)
    x_mask = Concatenate(axis=3)([x_mask,x_mask_ident_1])
    
    x_mask_ident_1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_mask_ident_1)
    x_mask_ident_2 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_mask)
    # 2nd layer group
    x_mask = Convolution2D(16, 3, 3, activation=None, border_mode='same', name='conv2_mask', subsample=(1, 1))(x_mask)
    x_mask = BatchNormalization()(x_mask)
    x_mask = ReLU()(x_mask)
    x_mask = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool2_mask')(x_mask)
    x_mask = Concatenate(axis=3)([x_mask,x_mask_ident_1,x_mask_ident_2])

    x_mask_ident_1 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_mask_ident_1)
    x_mask_ident_2 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_mask_ident_2)
    x_mask_ident_3 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid')(x_mask)
    # 3rd layer group
    x_mask = Convolution2D(32, 3, 3, activation=None, border_mode='same', name='conv3_mask', subsample=(1, 1))(x_mask)
    x_mask = BatchNormalization()(x_mask)
    x_mask = ReLU()(x_mask)
    x_mask = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', name='pool3_mask')(x_mask)
    x_mask = Concatenate(axis=3)([x_mask,x_mask_ident_1,x_mask_ident_2,x_mask_ident_3])

    x_mask = GlobalMaxPooling2D()(x_mask)
    x_mask = BatchNormalization(name="final_features_344_mask")(x_mask)
    
    ##################################################################################################################
    x = concatenate([x,x_mask])
        
    x = Dense(64, activation='relu', name="final_features_64")(x)
    out_class = Dense(5, activation='softmax', name='out_class')(x)

    model = Model(input=[inputs,inputs_mask], output=out_class)
    
    if load_weight_path is not None: model.load_weights(load_weight_path, by_name=False)

    optimizer = SGD(lr=0.001, momentum=0.9, nesterov=True)
    loss = {"out_class": "categorical_crossentropy"}
    metrics={"out_class": [categorical_accuracy, categorical_crossentropy]}

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.summary(line_length=140)

    return model